In [1]:
import numpy as np

import sys
sys.path.append("/Applications/KiCad/KiCad.app/Contents/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages")
import pcbnew

from wirebond_pcb import Board

In [2]:
board_width_mm = 35
board_height_mm = 35

board = Board(
    width_mm=board_width_mm,
    height_mm=board_height_mm,
)

# put the plated thru-holes on the edges for soldering
edge_offset_mm = 3
num_holes = 8
hole_centers_x_mm = np.linspace(edge_offset_mm, board_width_mm - edge_offset_mm, num_holes).tolist()
for x in hole_centers_x_mm:
    board.add_plated_thru_hole(1, (x, edge_offset_mm), 2)
    board.add_plated_thru_hole(1, (x, board_height_mm - edge_offset_mm), 2)

# place wirebonding pads near the chip
pad_size_mm = 0.5
pad_spacing_mm = 0.5
pad_array_width_mm = num_holes * (pad_size_mm + pad_spacing_mm)
pad_centers_x_mm = np.linspace(
    (board_width_mm - pad_array_width_mm) / 2,
    (board_width_mm + pad_array_width_mm) / 2,
    num_holes
).tolist()
chip_size_mm = 5.2
pad_chip_offset_y_mm = 2
pad_top_row_y_mm = (board_height_mm - chip_size_mm) / 2 - pad_chip_offset_y_mm
pad_bottom_row_y_mm = (board_height_mm + chip_size_mm) / 2 + pad_chip_offset_y_mm
for x in pad_centers_x_mm:
    board.add_wirebond_pad(
        width_mm=pad_size_mm,
        height_mm=pad_size_mm,
        center_x_mm=x,
        center_y_mm=pad_top_row_y_mm,
    )
    board.add_wirebond_pad(
        width_mm=pad_size_mm,
        height_mm=pad_size_mm,
        center_x_mm=x,
        center_y_mm=pad_bottom_row_y_mm,
    )

# Add traces connecting PTHs to wirebond pads
trace_width_mm = 0.3  # Reasonable width for low resistance

# Connect top row
for i, (pth_x, pad_x) in enumerate(zip(hole_centers_x_mm, pad_centers_x_mm)):
    # Top edge PTH to top wirebond pad
    board.add_trace(
        start_xy_mm=(pth_x, edge_offset_mm),
        end_xy_mm=(pad_x, pad_top_row_y_mm),
        width_mm=trace_width_mm
    )
    
    # Bottom edge PTH to bottom wirebond pad  
    board.add_trace(
        start_xy_mm=(pth_x, board_height_mm - edge_offset_mm),
        end_xy_mm=(pad_x, pad_bottom_row_y_mm),
        width_mm=trace_width_mm
    )

# Add silkscreen boxes around PTH pairs
box_padding_mm = 1.5  # Space around the PTHs

# Group PTHs into pairs and draw boxes
for i in range(0, num_holes, 2):  # Step by 2 to get pairs
    if i + 1 < num_holes:  # Make sure we have a pair
        # Top edge pair
        box_left = hole_centers_x_mm[i] - box_padding_mm
        box_right = hole_centers_x_mm[i + 1] + box_padding_mm
        box_top = edge_offset_mm - box_padding_mm
        box_bottom = edge_offset_mm + box_padding_mm
        board.add_silkscreen_box(box_left, box_top, box_right, box_bottom)
        
        # Bottom edge pair
        box_top = board_height_mm - edge_offset_mm - box_padding_mm
        box_bottom = board_height_mm - edge_offset_mm + box_padding_mm
        board.add_silkscreen_box(box_left, box_top, box_right, box_bottom)

# add text labels for each pair
for i in range(0, num_holes, 2):
    if i + 1 < num_holes:
        pair_num = i // 2 + 1
        text_x = (hole_centers_x_mm[i] + hole_centers_x_mm[i + 1]) / 2
        board.add_text_to_top_silkscreen(f"A{5-pair_num}", text_x, edge_offset_mm + 2.5, size_mm=2.5, rotation_degrees=180)
        board.add_text_to_top_silkscreen(f"B{pair_num}", text_x, board_height_mm - edge_offset_mm - 2.5, size_mm=2.5)

# add square outline to center of board to help positioning the chip
board.add_chip_outline_to_top_silkscreen(5.5, 5.5)

# add text identifying the board etc
board.add_text_to_top_silkscreen(
    "ITO electrical\nconnectivity\ntesting v1",
    x_mm=edge_offset_mm - 1,
    y_mm=board_height_mm / 2,
    size_mm=1.5,
    rotation_degrees=90,
    vertical_justify="top"
)
board.add_text_to_top_silkscreen(
    "Noyce ion trap\nJuly, 2025\nUC Davis\nR-Lab",
    x_mm=board_width_mm - edge_offset_mm + 1,
    y_mm=board_height_mm / 2,
    size_mm=1.5,
    rotation_degrees=-90,
    vertical_justify="top"
)

board.add_serial_number_location_to_bottom_silkscreen()

board.generate_gerbers(f"../output_files/example_gerber")

./src/common/stdpbase.cpp(59): assert ""traits"" failed in Get(): create wxApp before calling this
